In [6]:
%pip install xlrd==2.0.1

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: C:\Users\Elena\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [5]:
%pip install pywin32

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: C:\Users\Elena\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [7]:
%pip install unidecode

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: C:\Users\Elena\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [14]:
# Cargamos las librerias necesarias
import pandas as pd
from pathlib import Path
from win32com.client import DispatchEx
import unicodedata

In [11]:
# Directorio donde está el notebook
BASE_DIR = Path.cwd()
directorio = BASE_DIR / "01. Informe Instalaciones"

In [12]:
#_____________________________________________________________________________________
# 06.Potencia instalada del PRETOR
#_____________________________________________________________________________________


def convertir_xls_a_xlsx(carpeta_1: str) -> list[Path]:
    """
    Abre cada .xls con Excel (COM) y lo guarda como .xlsx en la misma carpeta.
    Requiere Windows + Excel instalado: pip install pywin32
    Devuelve la lista de rutas .xlsx generadas.
    """

    carpeta = Path(carpeta_1)
    xls_files = sorted(f for f in carpeta.glob("*.xls") if f.is_file())
    if not xls_files:
        return []

    excel = DispatchEx("Excel.Application")
    excel.Visible = False
    excel.DisplayAlerts = False

    generados = []
    try:
        for f in xls_files:
            destino = f.with_suffix(".xlsx")
            wb = excel.Workbooks.Open(str(f))
            # 51 = xlOpenXMLWorkbook (xlsx, sin macros)
            wb.SaveAs(str(destino), FileFormat=51)
            wb.Close(SaveChanges=False)
            generados.append(destino)
    finally:
        excel.Quit()
    return generados



def concat_excels(carpeta_1: str, carpeta_2: str) -> pd.DataFrame:
    carpeta = Path(carpeta_1)

    # 1) Convertir .xls -> .xlsx (si hay)
    try:
        convertir_xls_a_xlsx(carpeta)
    except Exception as e:
        print(f"⚠️ No se pudo convertir con Excel COM: {e}")

    # 2) Leer todos los Excel modernos
    archivos = sorted([f for f in carpeta.glob("*") if f.suffix.lower() in (".xlsx", ".xlsm")])
    if not archivos:
        raise FileNotFoundError("No hay .xlsx/.xlsm en la carpeta (ni conversión exitosa).")

    frames = []
    for f in archivos:
        df = pd.read_excel(f, engine="openpyxl")
        if not df.empty:
            df = df.copy()
            df["__archivo__"] = f.name
            frames.append(df)
    out = pd.concat(frames, ignore_index=True)
    
    # Normalizar nombres de columnas para eliminar acentos
    out.columns = [unicodedata.normalize("NFKD", c).encode("ascii", "ignore").decode("utf-8") for c in out.columns]
    cols_a_limpiar = out.columns[1:4]

    # Quitar acentos y caracteres especiales solo en esas columnas
    for col in cols_a_limpiar:
        out[col] = out[col].apply(
            lambda x: ''.join(
                ch for ch in unicodedata.normalize('NFKD', str(x))
                if not unicodedata.combining(ch)
            ) if isinstance(x, str) else x
        )
        
    out = out.rename(columns={'Nombre de Instalacion': 'Nombre',
                              'Municipio de la Instalacion': 'Municipio',
                              'Provincia de la Instalacion': 'Provincia',
                              'Potencia Instalada KW': 'Potencia',
                              'Grupo Normativo': 'Grupo',
                              'Fecha de Inscripcion Definitiva': 'Fecha'})
    
    out.to_excel(f"{carpeta_2}/PRETOR_TOTAL.xlsx", index=False, engine="openpyxl")
    # Crear el nuevo DataFrame con las columnas que te interesan
    out_red = out[[
        'Municipio', 'Potencia', 'Grupo', 'Fecha',
        'Provincia'
    ]].copy()  # copia para evitar advertencias de asignación


    # Aplicar al DataFrame
# --- 1) Mapear Grupo Normativo a Solar/Eolico ---
    grupo_map = {
    "b.1": 'Solar',
    "b.1.1": 'Solar',
    "b.1.2": 'Solar',
    "b.2": 'Eolico',
    "b.2.1": 'Eolico',
    "b.2.2": 'Eolico'
    }

    out_red["Grupo"] = out_red["Grupo"].map(grupo_map)
   
    
    # --- 2) Filtrar por Grupo y fecha válida ---
    # fecha no nula, no vacía, y distinta de "n/D" (cualquier capitalización)
    fecha_str = out_red['Fecha'].astype(str).str.strip()
    mask_fecha_valida = out_red['Fecha'].notna() & (fecha_str != '') & (fecha_str.str.lower() != 'n/d')

    mask_grupo_valido = out_red['Grupo'].isin(['Solar', 'Eolico'])

    out_red = out_red.loc[mask_grupo_valido & mask_fecha_valida].copy()

    # (Opcional) Parsear la fecha a datetime y descartar las que no se puedan parsear
    out_red['Fecha'] = pd.to_datetime(out_red['Fecha'], errors='coerce', dayfirst=True)
    out_red = out_red[out_red['Fecha'].notna()]


    out_red.to_excel(f"{carpeta_2}/PRETOR_REDUCIDO.xlsx", index=False, engine="openpyxl")
    out_red.to_csv(f"{carpeta_2}/PRETOR_REDUCIDO.csv", index=False)
    
    return out



In [ ]:
convertir_xls_a_xlsx(directorio)

In [ ]:
df_out= concat_excels(carpeta_1=directorio, carpeta_2=BASE_DIR)